## Objectives

Note: Product Description is the common column between import sheet and data bank sheet.
Objective – model number and brand name columns are required in import sheet. Create a column where % match (eg 100% or 50%, etc.) between product description column of both table is represented with their percentage match.

Task 1 – Return Brand Name and Model Name in import table from Data bank table where product description of import file is 100% match with product description of Data Bank. 

Task 2 - Return Brand Name and Model name column in import table from Data bank table where product description of import file is up to 90% match with product description of Data Bank. (Consider only those product descriptions of import table for which Brand Name and Model name column is not returned in task 1).
Task 3 – For rest rows of import table where Brand Name and Model name column is not returned from task1 and task 2, find Brand Name from ‘Shipper Name’ column and find it in Brand table.
Do necessary Data cleaning before doing tasks.
												
															
															
															

In [3]:
import numpy as np
from scipy.stats import t
import matplotlib.pyplot as plt
import plotly.express as px

import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\Satish\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
pip install fuzzywuzzy


  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)


In [6]:
data_bank_file = "data bank.xlsx"
import_file = "import.xlsx"
data_bank_df = pd.read_excel(data_bank_file, engine="openpyxl")
import_df = pd.read_excel(import_file, engine="openpyxl")


In [12]:
# Task 1: Exact 100% Match
exact_matches = pd.merge(import_df, data_bank_df, on="Product Description", how="inner")[["Product Description","Brand Name", "Model Name"]]
task1_results = exact_matches.drop_duplicates()
task1_results

,Product Description,Brand Name,Model Name
0,16641210 FUJINON LENS F XF16MMF1.4 R WR PH,Fujifilm,XF16 mm
2,16792576 FUJINON LENS F XF30MMF2.8 R LM WR MC TH,Fujifilm,XF30 mm
10,16780028 FUJINON LENS F XF56MMF1.2 R WR TH,Fujifilm,F XF56 mm
106,16607446 FUJINON LENS XF55-200MMF3.5-4.8 R LM ...,Fujifilm,XF55-200 mm
122,5137C012AA EOS R7 RFS18150mm WIFI# DS126841 BI...,Canon,EOS R7
...,...,...,...
2795,32852720 ZV-E10/BQ IN5 DIGITAL STILL CAMERA ZV...,Sony,ZV-E10
2796,ILCE-7M3/BQ IN5 32850420 DIGITAL STILL CAMERA ...,Sony,ILCE-7M3
2797,ZV-E10/BQ IN5 32852720 DIGITAL STILL CAMERA WW...,Sony,ZV-E10
2798,SRG-X120/BC IN5 32495790 COLOR CMOS TELEVISION...,Sony,SRG-X120/BC


In [ ]:
# Task 2: Up to 90% Match
remaining_import_df = import_df[~import_df["Product Description"].isin(task1_results["Product Description"])]

similar_matches = []

for import_desc in remaining_import_df["Product Description"]:
    # Finding the best match with a similarity score >= 90%
    _ , match, score = process.extractOne(import_desc, data_bank_df["Product Description"], scorer=fuzz.token_sort_ratio)
    if score >= 90:
        matched_row = data_bank_df[data_bank_df["Product Description"] == match][["Brand Name", "Model Name"]]
        similar_matches.append(matched_row)

# Concatenate the matched rows
task2_results = pd.concat(similar_matches)

# Task 3: Find Brand Name from 'Shipper Name'
remaining_import_df["Brand Name"] = remaining_import_df["Shipper Name"].apply(lambda shipper: process.extractOne(shipper, brand_df["Brand Name"], scorer=fuzz.token_sort_ratio)[0])

# Assuming the 'Brand' table is available in the 'brand_df' dataframe
# Replace 'brand_df' with the actual dataframe containing the 'Brand' table

# Concatenate the results of all tasks
final_results = pd.concat([task1_results, task2_results, remaining_import_df[["Brand Name", "Model Name"]]])

# Save the final results to a new Excel file
final_results.to_excel("Final_Results.xlsx", index=False, engine="openpyxl")


In [2]:
import pandas as pd
from fuzzywuzzy import fuzz, process

# Read the data from Excel files
data_bank_df = pd.read_excel("data bank.xlsx")
import_df = pd.read_excel("import.xlsx")

# Task 1: 100% Match
task1_results = import_df.merge(data_bank_df, on="Product Description", how="inner")[["Brand Name", "Model Name"]]
task1_results["Match Percentage"] = 100

# Task 2: Up to 90% Match
remaining_import_df = import_df[~import_df["Product Description"].isin(task1_results["Product Description"])]

# Finding the best match with a similarity score >= 90% for each "Product Description"
similar_matches = remaining_import_df["Product Description"].apply(lambda desc: process.extractOne(desc, data_bank_df["Product Description"], scorer=fuzz.partial_token_sort_ratio))
similar_matches = similar_matches[similar_matches.apply(lambda x: x[1] >= 90)]

# Extracting the matching rows from data_bank_df
matched_indices = similar_matches.index
task2_results = data_bank_df.iloc[similar_matches.apply(lambda x: x[2])][["Brand Name", "Model Name"]]

# Task 3: Find Brand Name from 'Shipper Name'
brand_df = data_bank_df[["Brand Name"]].drop_duplicates(subset="Brand Name")
remaining_import_df["Brand Name"] = remaining_import_df["Shipper Name"].apply(lambda shipper: process.extractOne(shipper, brand_df["Brand Name"], scorer=fuzz.partial_token_sort_ratio)[0])

# Concatenate the results of all tasks
final_results = pd.concat([task1_results, task2_results, remaining_import_df[["Brand Name", "Model Name"]]])

# Save the final results to a new Excel file
final_results.to_excel("Final_Results.xlsx", index=False, engine="openpyxl")


KeyError: 'Product Description'